In [ ]:
import cv2
import numpy as np
import os

def process_images_to_bw(
    main_folder='inked',
    output_main_folder='inked_output',
    threshold_value=50
):
    """
    This script:
    1. Reads images from each subfolder in 'inked'.
    2. Converts them to grayscale.
    3. Applies a binary threshold (THRESH_BINARY_INV).
    4. Performs a morphological open operation to clean small noise.
    5. Saves a purely black-and-white image (white pen markings on black background)
       in a separate folder 'inked_output' outside the 'inked' folder, mirroring subfolder structure.
    """
    # Ensure the output main folder exists (outside 'inked')
    os.makedirs(output_main_folder, exist_ok=True)

    # Iterate over each subfolder in the main folder
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)
        if os.path.isdir(subfolder_path):
            # Create a corresponding subfolder in the output directory
            output_subfolder_path = os.path.join(output_main_folder, subfolder)
            os.makedirs(output_subfolder_path, exist_ok=True)

            # Process each image in the subfolder
            for filename in os.listdir(subfolder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(subfolder_path, filename)
                    img = cv2.imread(image_path)
                    if img is None:
                        print(f"Could not read image: {image_path}")
                        continue

                    # 1. Convert image to grayscale
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                    # 2. Apply thresholding (threshold_value is adjustable; default=50)
                    #    THRESH_BINARY_INV -> Dark markings become white, background black
                    _, mask = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

                    # 3. Morphological operation (Opening) to remove small noise
                    kernel = np.ones((3, 3), np.uint8)
                    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

                    # (Optional) Invert if you want black markings on a white background:
                    # mask_cleaned = cv2.bitwise_not(mask_cleaned)

                    # 4. Save the purely black-and-white result in the new output folder
                    output_path = os.path.join(output_subfolder_path, f'pen_{filename}')
                    cv2.imwrite(output_path, mask_cleaned)
                    print(f'Processed and saved: {output_path}')

if __name__ == "__main__":
    # Usage example:
    #  - The original images are in a folder named "inked"
    #  - The processed images will be saved in a folder named "inked_output"
    #  - You can also adjust the threshold_value if needed.
    process_images_to_bw(main_folder="inked", output_main_folder="masks", threshold_value=50)


: 